<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AAI_FP_GROK4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_core -q
!pip install langchain -q
!pip install xai-sdk -q
!pip install langgraph -q

!pip install langchain_xai -q

!pip install -U langchain-community -q

In [1]:
import datetime
import random
import time
import json
from datetime import timezone, timedelta
from typing import List, Dict, Any, Union, Optional

# --- Corrected Pydantic Imports ---
from pydantic import BaseModel, Field # Use direct pydantic import as per warning
import operator # Still needed for Annotated typing if used, but for now we simplify

# --- XAI SDK Imports and Langchain specific imports ---
from xai_sdk import Client
from google.colab import userdata

# Langchain specific imports for ChatXAI and tool use
from langchain_xai import ChatXAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain_core.runnables import RunnablePassthrough # Added for mapping inputs
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser
from langchain.agents.format_scratchpad.tools import format_to_tool_messages # Correct utility for agent_scratchpad


# --- Configuration ---
try:
    XAI_API_KEY = userdata.get('XAI_KEY')
    if XAI_API_KEY:
        print("XAI_KEY found in Colab secrets.")
    else:
        print("WARNING: XAI_KEY not found in Colab secrets. Please ensure it is set.")
except ImportError:
    import os
    print("WARNING: Not running in Google Colab. Attempting to get XAI_KEY from environment variables.")
    XAI_API_KEY = os.environ.get('XAI_KEY')
    if XAI_API_KEY is None:
        print("WARNING: XAI_KEY not found in environment variables. Cannot run LLM-driven agent.")

# --- Mock External Tools (Simulating APIs/Databases) ---
class WeatherToolInput(BaseModel):
    location_icao: str = Field(description="The ICAO code of the airport (e.g., 'CYUL').")

@tool("WeatherTool_get_weather", args_schema=WeatherToolInput)
def get_weather(location_icao: str) -> Dict[str, Any]:
    """Fetches current weather information for a given airport ICAO code."""
    print(f"  [Tool Call] Fetching weather for {location_icao}...")
    if "CYUL" in location_icao: # Montreal
        return {"icao": location_icao, "temp_c": 25, "wind_knots": 10, "conditions": "Clear", "visibility_sm": 10}
    elif "CYVR" in location_icao: # Vancouver
        return {"icao": location_icao, "temp_c": 18, "wind_knots": 15, "conditions": "Partly Cloudy", "visibility_sm": 8}
    elif "EHAM" in location_icao: # Amsterdam
        return {"icao": location_icao, "temp_c": 15, "wind_knots": 20, "conditions": "Rain", "visibility_sm": 3}
    elif "KLGA" in location_icao: # LaGuardia
        return {"icao": location_icao, "temp_c": 22, "wind_knots": 12, "conditions": "Scattered Clouds", "visibility_sm": 10}
    elif "KLAX" in location_icao: # Los Angeles
        return {"icao": location_icao, "temp_c": 13, "wind_knots": 10, "conditions": "Varies", "visibility_sm": 10}
    elif "KJFK" in location_icao: # JFK
        return {"icao": location_icao, "temp_c": 18, "wind_knots": 23, "conditions": "Varies", "visibility_sm": 9}
    return {"icao": location_icao, "temp_c": random.randint(5, 30), "wind_knots": random.randint(5, 25), "conditions": "Varies", "visibility_sm": random.randint(5, 10)}

class NotamToolInput(BaseModel):
    location_icao: Optional[str] = Field(None, description="Optional: ICAO code of an airport.")
    route_points: Optional[str] = Field(None, description="Optional: String representing the flight route (e.g., 'CYUL DCT WAYPT1 DCT CYVR').")

@tool("NotamTool_get_notams", args_schema=NotamToolInput)
def get_notams(location_icao: Optional[str] = None, route_points: Optional[str] = None) -> List[str]:
    """Retrieves NOTAMs (Notices to Airmen) for an airport or along a flight route."""
    print(f"  [Tool Call] Fetching NOTAMs for {location_icao if location_icao else 'route'}...")
    notams = []
    if location_icao:
        if "CYUL" in location_icao:
            notams.append("CYUL RWY 24L/06R closed for maintenance until 2507201200Z.")
        if "CYVR" in location_icao:
            notams.append("CYVR APP freq 121.1 out of service. Use 121.3.")
        if "EHAM" in location_icao:
            notams.append("EHAM Taxiway B closed. Expect delays.")
        if "KLGA" in location_icao:
            notams.append("KLGA ATC advisories for heavy traffic volume. Expect holding.")

    if route_points and "WAYPT_SOUTH" in route_points:
        notams.append("TEMPORARY FLIGHT RESTRICTION due to military exercise, FL300-400, 2507191400Z-1800Z.")

    if not notams and (location_icao or route_points):
        notams.append(f"No specific active NOTAMs found for {location_icao if location_icao else 'the specified route'}.")
    elif not notams:
        notams.append("No NOTAMs requested or found.")

    return notams

class RouteOptimizerToolInput(BaseModel):
    origin: str
    destination: str
    preferred_altitude_fl: int
    weather_data: Dict[str, Any] = Field(description="Dictionary of weather data for relevant airports (e.g., {'CYUL': {...}, 'CYVR': {...}}).")
    notam_data: List[str] = Field(description="List of relevant NOTAM strings.")
    aircraft_type: str

@tool("RouteOptimizerTool_optimize_route", args_schema=RouteOptimizerToolInput)
def optimize_route(origin: str, destination: str, preferred_altitude_fl: int, weather_data: Dict[str, Any], notam_data: List[str], aircraft_type: str) -> Dict[str, Any]:
    """Optimizes a flight route considering origin, destination, altitude, weather, and NOTAMs."""
    print(f"  [Tool Call] Optimizing route for {aircraft_type} from {origin} to {destination}...")
    base_route = f"{origin} DCT WAYPT1 DCT WAYPT2 DCT {destination}"
    flight_time_hours = random.uniform(5, 10)
    distance_nm = random.randint(2000, 4000)

    if "military exercise" in " ".join(notam_data) and preferred_altitude_fl > 300:
        print("    [RouteOptimizer] Adjusting route due to high-altitude military exercise NOTAM.")
        base_route = f"{origin} DCT WAYPT_SOUTH DCT WAYPT_NORTH DCT {destination} (rerouted)"
        flight_time_hours += 1.5
        distance_nm += 300

    if "Rain" in weather_data.get(destination, {}).get("conditions", ""):
        print("    [RouteOptimizer] Suggesting alternate due to heavy rain at destination.")
        alternate_icao = "CYYZ" if destination == "EHAM" else "KSEA"
        base_route += f" ALT {alternate_icao}"

    return {
        "route_string": base_route,
        "distance_nm": distance_nm,
        "estimated_flight_time_hours": round(flight_time_hours, 2),
        "preferred_altitude_fl": preferred_altitude_fl
    }

class FuelCalculatorToolInput(BaseModel):
    aircraft_type: str
    distance_nm: Union[int, float]
    flight_time_hours: Union[int, float]
    conditions: str = Field(description="Weather conditions at destination (e.g., 'Rain', 'Clear').")
    destination_wind_knots: int = Field(description="Wind speed in knots at destination.")

@tool("FuelCalculatorTool_calculate_fuel", args_schema=FuelCalculatorToolInput)
def calculate_fuel(aircraft_type: str, distance_nm: Union[int, float], flight_time_hours: Union[int, float], conditions: str, destination_wind_knots: int) -> Dict[str, Any]:
    """Calculates the required fuel for a flight."""
    print(f"  [Tool Call] Calculating fuel for {aircraft_type}...")
    base_fuel_burn_per_hour = 3000 # kg/hr for a large jet
    contingency_factor = 1.15 # 15% for contingency
    extra_for_conditions = 0

    if "Rain" in conditions or destination_wind_knots > 20:
        extra_for_conditions = 500 # kg for adverse conditions

    fuel_needed_kg = (base_fuel_burn_per_hour * flight_time_hours * contingency_factor) + extra_for_conditions
    return {"fuel_needed_kg": round(fuel_needed_kg, 0)}

# Collect all tools
tools = [get_weather, get_notams, optimize_route, calculate_fuel]

# --- The LLM-driven Agent Core (now using Langchain AgentExecutor) ---
class LLMAgent:
    def __init__(self, api_key: str, verbose: bool = True):
        self.verbose = verbose
        if not api_key:
            raise ValueError("API key must be provided for LLMAgent.")

        self.llm = ChatXAI(model="grok-4", xai_api_key=api_key, temperature=0) # temperature=0 for consistent planning

        # Bind tools to the LLM. This is how Langchain tells the LLM about available tools.
        self.llm_with_tools = self.llm.bind_tools(tools)

        # Define the prompt template for the agent
        self.prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "You are a highly intelligent flight planning assistant. Your goal is to plan flights based on user requests, using the available tools. You must use the tools to gather information and make calculations. After gathering all necessary information, provide a comprehensive flight briefing. Always use the tools to get the required information."),
                MessagesPlaceholder(variable_name="chat_history"), # For ongoing conversation memory
                ("human", "{input}"), # For the current user query
                MessagesPlaceholder(variable_name="agent_scratchpad"), # CRITICAL: For agent's internal thought/tool use
            ]
        )

        # Create the Langchain AgentExecutor
        self.agent_executor = AgentExecutor(
            # The chain now correctly maps intermediate_steps to agent_scratchpad
            agent=(
                RunnablePassthrough.assign(
                    agent_scratchpad=lambda x: format_to_tool_messages(x["intermediate_steps"])
                )
                | self.prompt
                | self.llm_with_tools
                | ToolsAgentOutputParser()
            ),
            tools=tools,
            verbose=self.verbose,
            handle_parsing_errors=True
        )
        self._log("LLMAgent initialized with Langchain AgentExecutor.")

    def _log(self, message):
        if self.verbose:
            print(f"[LLMAgent] {message}")

    def run_inference(self, user_query: str, initial_context: dict) -> str:
        self._log("Starting inference run for new query.")

        # Format initial context into a human-readable message for the LLM
        context_message = f"Initial planning context: {initial_context}"

        # Langchain AgentExecutor works with a list of messages for chat_history
        # The first message sets the initial context. The user_query is passed separately as 'input'.
        chat_history = [
            HumanMessage(content=context_message)
        ]

        try:
            # Invoke the AgentExecutor with the current chat history and input
            # The AgentExecutor handles the multi-turn interaction with the LLM and tool execution internally
            result = self.agent_executor.invoke({"input": user_query, "chat_history": chat_history})

            final_response_text = result.get("output", "No specific output from the agent.")
            self._log("AgentExecutor finished.")
            return final_response_text

        except Exception as e:
            self._log(f"An error occurred during agent execution: {type(e).__name__}: {e}")
            # More specific error handling for prompt issues
            if "Input to ChatPromptTemplate is missing variables" in str(e):
                return f"An internal configuration error occurred: {e}"
            return f"An internal error occurred during planning: {type(e).__name__}: {e}"


# --- Modified FlightPlanningAgent to leverage Langchain LLMAgent ---
class FlightPlanningAgent:
    def __init__(self, api_key: str, name: str = "Flight Planner (Grok-4)", verbose: bool = True):
        self.name = name
        self.verbose = verbose
        self.llm_agent = LLMAgent(api_key=api_key, verbose=verbose)
        self.history = []

    def _log(self, message):
        if self.verbose:
            print(f"[{self.name}] {message}")
        self.history.append(message)

    def plan_flight(self, origin_icao: str, destination_icao: str, aircraft_type: str, preferred_altitude_fl: int = 350) -> str:
        user_request = f"Plan a flight for a {aircraft_type} from {origin_icao} to {destination_icao} at FL{preferred_altitude_fl}. Provide a detailed flight briefing including weather, NOTAMs, route, and fuel."
        self._log(f"Received user request: '{user_request}'")

        # Define the EST timezone object (UTC-5)
        est_timezone = timezone(timedelta(hours=-5))
        # Updated to current time as per your context: Saturday, July 19, 2025 at 3:46:30 PM EDT.
        current_datetime_est = datetime.datetime(2025, 7, 19, 15, 46, 30, tzinfo=est_timezone)

        initial_context = {
            "origin": origin_icao,
            "destination": destination_icao,
            "aircraft_type": aircraft_type,
            "preferred_altitude_fl": preferred_altitude_fl,
            "current_time_est": current_datetime_est.strftime("%Y-%m-%d %H:%M:%S EST")
        }

        final_briefing_response = self.llm_agent.run_inference(user_request, initial_context)

        self._log("Flight planning process driven by Grok-4 complete.")
        return final_briefing_response

# --- Demonstrate Usage ---
if __name__ == "__main__":
    print("--- Demonstrating Core FlightPlanningAgent (incorporating Reflection, Tool Use, Planning) ---")
    print("--- Powered by Grok-4 LLM (using Langchain's ChatXAI) ---")

    if not XAI_API_KEY:
        print("\nSkipping LLM-driven demonstration as XAI_API_KEY was not found.")
        print("Please ensure your 'XAI_KEY' is correctly set in Google Colab secrets or environment variables.")
    else:
        print("\n--- Scenario 1: Standard Flight (Montreal to Vancouver) ---")
        flight_planner_agent_s1 = FlightPlanningAgent(api_key=XAI_API_KEY, verbose=False)
        briefing1 = flight_planner_agent_s1.plan_flight("CYUL", "CYVR", "Boeing 787", preferred_altitude_fl=350)
        print("\nFinal Briefing from LLM-driven Agent (Scenario 1):")
        print(briefing1)

        print("\n" + "="*80 + "\n")

        print("--- Scenario 2: Flight with potential issues (Montreal to Amsterdam) ---")
        flight_planner_agent_s2 = FlightPlanningAgent(api_key=XAI_API_KEY, verbose=False)
        briefing2 = flight_planner_agent_s2.plan_flight("CYUL", "EHAM", "Airbus A350", preferred_altitude_fl=380)
        print("\nFinal Briefing from LLM-driven Agent (Scenario 2):")
        print(briefing2)

        print("\n" + "="*80 + "\n")

        print("--- Scenario 3: Another Standard Flight (Vancouver to New York - LaGuardia) ---")
        flight_planner_agent_s3 = FlightPlanningAgent(api_key=XAI_API_KEY, verbose=False)
        briefing3 = flight_planner_agent_s3.plan_flight("CYVR", "KLGA", "Cessna Citation", preferred_altitude_fl=250)
        print("\nFinal Briefing from LLM-driven Agent (Scenario 3):")
        print(briefing3)

        print("\n" + "="*80 + "\n")

        print("--- Scenario 4: Flight with expected military exercise NOTAM (KLAX to KJFK) ---")
        flight_planner_agent_s4 = FlightPlanningAgent(api_key=XAI_API_KEY, verbose=False)
        briefing4 = flight_planner_agent_s4.plan_flight("KLAX", "KJFK", "Boeing 737", preferred_altitude_fl=350)
        print("\nFinal Briefing from LLM-driven Agent (Scenario 4):")
        print(briefing4)

XAI_KEY found in Colab secrets.
--- Demonstrating Core FlightPlanningAgent (incorporating Reflection, Tool Use, Planning) ---
--- Powered by Grok-4 LLM (using Langchain's ChatXAI) ---

--- Scenario 1: Standard Flight (Montreal to Vancouver) ---
  [Tool Call] Fetching weather for CYUL...
  [Tool Call] Fetching weather for CYVR...
  [Tool Call] Fetching NOTAMs for route...
  [Tool Call] Optimizing route for Boeing 787 from CYUL to CYVR...
  [Tool Call] Calculating fuel for Boeing 787...

Final Briefing from LLM-driven Agent (Scenario 1):
### Flight Briefing: CYUL to CYVR

**Flight Overview**  
- **Origin Airport**: CYUL (Montréal–Pierre Elliott Trudeau International Airport, Montreal, Canada)  
- **Destination Airport**: CYVR (Vancouver International Airport, Vancouver, Canada)  
- **Aircraft Type**: Boeing 787  
- **Preferred Cruising Altitude**: FL350 (35,000 feet)  
- **Planning Time**: 2025-07-19 15:46:30 EST (Note: All data is based on conditions at or near this time; real-time upda